In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


#Scraping
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

#BNN
import numpy as np
from sklearn import datasets

import torch
import torch.nn as nn
import torch.optim as optim

import torchbnn as bnn

In [ ]:
# Load .env file
load_dotenv('../.env.local')  # Adjust the relative path according to your directory structure

# Get the project path from environment variables
project_path = os.getenv('MY_PROJECT_PATH')

# Read CSV into DataFrame
df = pd.read_csv(project_path)

In [ ]:
# Display the DataFrame
df.head()

In [ ]:
url = "https://u.gg/lol/profile/euw1/thehighground/champion-stats"

options = Options()
options.add_argument("--headless")
options.binary_location = "C:\\Users\\nicol\\Downloads\\chrome-win64\\chrome-win64\\chrome.exe"  # Path to your Chrome executable

service = Service("C:\\Users\\nicol\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

data = []  # List to store each row of data

while True:
    # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    
    # Scroll back to the top of the page
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(1)
    
    # Check if all rows have been loaded
    rows = driver.find_elements(By.CSS_SELECTOR, "div.rt-tr-group")
    if not rows or len(data) == len(rows):  # Adjusted condition to check data length instead of champions length
        break
    
    # Otherwise, continue to extract the data
    for i in range(len(data), len(rows)):
        row = rows[i]
        try:
            name = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(2)").text.strip()
            rank = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(1)").text.strip()
            champion = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(3)").get_attribute("textContent")
            winrate_stats = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(4)").text.strip()
            win_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(5)").text.strip()
            pick_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(7)").text.strip()
            ban_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(6)").text.strip()
            matches = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(8)").text.strip()
    
            # Append the extracted data to the data list
            data.append([rank, name, champion, winrate_stats, win_rate, pick_rate,ban_rate, matches])
        
        except Exception as e:
            print(f"Error in row {i}: {e}")


# Convert the data list into a pandas DataFrame
columns = ['Rank', 'Name', 'WinRate', 'KDA', 'Win rate', 'Pick Rate', 'Ban Rate', 'CS']
df_scraped = pd.DataFrame(data, columns=columns)

driver.quit()

df_scraped.head()

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time

url = "https://u.gg/lol/profile/euw1/thehighground/champion-stats"

options = Options()
options.add_argument("--headless")
options.binary_location = "C:\\Users\\nicol\\Downloads\\chrome-win64\\chrome-win64\\chrome.exe"

service = Service("C:\\Users\\nicol\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

# ... (your existing imports and setup code)

# Initialize list to store each row of data
data = []

# Get all the rows
rows = driver.find_elements(By.CSS_SELECTOR, "div.rt-tr-group")

for i in range(1, len(rows) + 1):  # start from 1 because CSS nth-child starts from 1
    try:
        # Create CSS selectors based on the row number for each column
        selectors = {
            'Champion': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(2)",
            'WinRate': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > span:nth-child(3)",
            'StrongRate': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > strong:nth-child(1)",
            'KDA': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > strong:nth-child(1)",
            'LP': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(5) > span:nth-child(1)",
            'MaxKills': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(6) > span:nth-child(1)",
            'MaxDeaths': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(7)",
            'CS': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(8) > span:nth-child(1)",
            'Damage': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(9) > span:nth-child(1)",
            'Gold': f"div.rt-tr-group:nth-child({i}) > div:nth-child(1) > div:nth-child(10) > span:nth-child(1)"
        }
        
        # Find the elements and extract the text
        row_data = []
        for key, selector in selectors.items():
            element = driver.find_element(By.CSS_SELECTOR, selector)
            text = element.text.strip() if element.text else 'N/A'
            row_data.append(text)
        
        # Append to data list
        data.append(row_data)
        
    except Exception as e:
        print(f"Error in row {i}: {e}")

# Convert the data to a DataFrame
columns = ['Champion', 'WinsLoses', 'Winrate', 'KDA', 'LP', 'MaxKills', 'MaxDeaths', 'CS', 'Damage', 'Gold']
df_individual = pd.DataFrame(data, columns=columns)

# Close the driver
driver.quit()

# Display the DataFrame (you could also save it to a file)
df_individual.head()

In [ ]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

# Step 1: Data Preprocessing
# Select features and target
feature_columns = ['Tier', 'Win rate', 'Pick Rate', 'Ban Rate', 'Matches', 'gameDuration', 'championId', 'summonerLevel', 'teamId']
target_column = 'win'

# Handle categorical variables and fill NaNs
df[feature_columns] = df[feature_columns].apply(lambda x: x.astype('category').cat.codes if x.dtype == 'object' else x)
df[feature_columns] = df[feature_columns].fillna(df[feature_columns].mean())

# Split data into training and testing sets
X = df[feature_columns].values
Y = df[target_column].values

# Scale the features
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

# Convert to PyTorch tensors
x, y = torch.from_numpy(X_scaled).float(), torch.from_numpy(Y).long()

# Step 2: Define Model
model = nn.Sequential(
    nn.Linear(in_features=len(feature_columns), out_features=100),
    nn.ReLU(),
    nn.Linear(in_features=100, out_features=2)
)

ce_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Step 3: Train Model
for step in range(10000):
    pre = model(x)
    ce = ce_loss(pre, y)
    
    optimizer.zero_grad()
    ce.backward()
    optimizer.step()

_, predicted = torch.max(pre.data, 1)
total = y.size(0)
correct = (predicted == y).sum()
print('- Accuracy: %f %%' % (100 * float(correct) / total))
print('- CE : %2.2f' % (ce.item()))

# Step 4: Test Model
# ... (You can use your own visualization method similar to draw_plot function)


In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    
plot_confusion_matrix(y.numpy(), predicted.numpy())

In [ ]:
model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=len(feature_columns), out_features=100),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=100, out_features=2),
)

# Loss and Optimizer
ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train Model
for step in range(3000):
    pre = model(x)
    ce = ce_loss(pre, y)
    kl = kl_loss(model)
    cost = ce + 0.1 * kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

_, predicted = torch.max(pre.data, 1)
total = y.size(0)
correct = (predicted == y).sum()
print('- Accuracy: %f %%' % (100 * float(correct) / total))
print('- CE : %2.2f, KL : %2.2f' % (ce.item(), kl.item()))

# Confusion Matrix Function
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.2)
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', cbar=False)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Visualization
plot_confusion_matrix(y.numpy(), predicted.numpy())

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL
url = "https://u.gg/lol/profile/euw1/thehighground/champion-stats"

options = Options()
options.add_argument("--headless")
options.binary_location = "C:\\Users\\nicol\\Downloads\\chrome-win64\\chrome-win64\\chrome.exe"

service = Service("C:\\Users\\nicol\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

# Initialize list to store each row of data
data = []

# Get page source and create BeautifulSoup object
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the driver
driver.quit()

# Get all the rows using BeautifulSoup
rows = soup.select("div.rt-tr-group")

for i, row in enumerate(rows, 1):  # start from 1 because CSS nth-child starts from 1
    try:
        selectors = {
            'Champion': "div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(2)",
            'WinsLoses': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > span:nth-child(3)",
            'Winrate': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > strong:nth-child(1)",
            'KDA': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > strong:nth-child(1)",
            'KillsDeathsAssists': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > span:nth-child(2)",  # New field
            'LP': "div:nth-child(1) > div:nth-child(5) > span:nth-child(1)",
            'MaxKills': "div:nth-child(1) > div:nth-child(6) > span:nth-child(1)",
            'MaxDeaths': "div:nth-child(1) > div:nth-child(7)",
            'CS': "div:nth-child(1) > div:nth-child(8) > span:nth-child(1)",
            'Damage': "div:nth-child(1) > div:nth-child(9) > span:nth-child(1)",
            'Gold': "div:nth-child(1) > div:nth-child(10) > span:nth-child(1)"
        }
        
        row_data = []
        for key, selector in selectors.items():
            element = row.select_one(selector)
            text = element.get_text(strip=True) if element else 'N/A'
            row_data.append(text)
        
        data.append(row_data)
        
    except Exception as e:
        print(f"Error in row {i}: {e}")

# Convert the data to a DataFrame
columns = ['Champion', 'WinsLoses', 'Winrate', 'KDA', 'KillsDeathsAssists', 'LP', 'MaxKills', 'MaxDeaths', 'CS', 'Damage', 'Gold']
df_individual = pd.DataFrame(data, columns=columns)

# Display the DataFrame
df_individual.head()

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL
url = "https://u.gg/lol/profile/euw1/leaguify/champion-stats?queueType=normal_draft_5x5"

options = Options()
options.add_argument("--headless")
options.binary_location = "C:\\Users\\nicol\\Downloads\\chrome-win64\\chrome-win64\\chrome.exe"

service = Service("C:\\Users\\nicol\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

driver.get(url)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

# Initialize list to store each row of data
data = []

# Get page source and create BeautifulSoup object
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Close the driver
driver.quit()

# Get all the rows using BeautifulSoup
rows = soup.select("div.rt-tr-group")

for i, row in enumerate(rows, 1):  # start from 1 because CSS nth-child starts from 1
    try:
        selectors = {
            'Champion': "div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(2)",
            'WinsLoses': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > span:nth-child(3)",
            'Winrate': "div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > strong:nth-child(1)",
            'KDA': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > strong:nth-child(1)",
            'KillsDeathsAssists': "div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > span:nth-child(2)",
            'LP': "div:nth-child(1) > div:nth-child(5) > span:nth-child(1)",
            'MaxKills': "div:nth-child(1) > div:nth-child(6) > span:nth-child(1)",
            'MaxDeaths': "div:nth-child(1) > div:nth-child(7)",
            'CS': "div:nth-child(1) > div:nth-child(8) > span:nth-child(1)",
            'Damage': "div:nth-child(1) > div:nth-child(9) > span:nth-child(1)",
            'Gold': "div:nth-child(1) > div:nth-child(10) > span:nth-child(1)"
        }
        
        row_data = []
        for key, selector in selectors.items():
            element = row.select_one(selector)
            text = element.get_text(strip=True) if element else 'N/A'
            row_data.append(text)
        
        data.append(row_data)
        
    except Exception as e:
        print(f"Error in row {i}: {e}")

# Convert the data to a DataFrame
columns = ['Champion', 'WinsLoses', 'Winrate', 'KDA', 'KillsDeathsAssists', 'LP', 'MaxKills', 'MaxDeaths', 'CS', 'Damage', 'Gold']
df_individual = pd.DataFrame(data, columns=columns)

In [ ]:
df_individual.head(200)